In [1]:
# Convierte de CSV a NetCDF.

import os
import sys

import numpy as np
import pandas as pd

import xarray as xr

# Datos de entrada
path_d = "/Volumes/DATA/temp/temp/NSRDB_2km/CSV/"
path_r = "/Volumes/DATA/temp/temp/NSRDB_2km/NetCDF/"
files = os.listdir(path_d)
if ".DS_Store" in files: files.remove(".DS_Store")

In [2]:
f="18.08_-99.99.csv"
lat = f[:5]
lon = f[6:-4]
print(f" Procesando coordenadas {lat}°N {lon}°W...    ", end = "\r")

if not os.path.exists(f"{path_r}{lat}_{lon}.nc"):
    df = pd.read_csv( path_d + f, index_col = "time",
        parse_dates = True ).reset_index()

    # Convertimos a Dataset.
    df["lat" ] = float(lat)
    df["lon"] = float(lon)
    ds = df.set_index( ["time", "lat", "lon"] ).astype( float
        ).round( decimals = 1 ).to_xarray()
    ds["lat"] = ds["lat"].assign_attrs( standard_name = "latitude",
            long_name = "Latitude", units = "degrees" )
    ds["lon"] = ds["lon"].assign_attrs( standard_name = "longitude",
            long_name = "Longitude", units = "degrees" )
    ds.to_netcdf( f"{path_r}{lat}_{lon}.nc" )

In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import os

path = "/Volumes/DATA/temp/temp/NSRDB_2km/NetCDF_n/"
path_r = "/Volumes/DATA/data/NSRDB_2km.nc"
files = os.listdir(path)

if ".DS_Store" in files: files.remove(".DS_Store")
ds = []
for f in files: ds.append( xr.open_dataset(path + f) )
xr.concat(ds, dim = "lat").to_netcdf(path_r)